In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf

In [3]:
ticker = 'SPY'
data = yf.download(ticker, start='2022-01-01', end='2025-03-15')['Close']['SPY']

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


In [5]:
daily_returns = data.pct_change().dropna()
sigma = daily_returns.std() * np.sqrt(252)

In [6]:
daily_returns.tail()

Date
2025-03-10   -0.026636
2025-03-11   -0.008313
2025-03-12    0.005307
2025-03-13   -0.013331
2025-03-14    0.020656
Name: SPY, dtype: float64

In [8]:
## PARAMETROS

#Precio Spot
S0 = data.iloc[-1]
sigma = daily_returns.std() * np.sqrt(252)

# Tasa libre de riesgo
r = 0.10

# Días al vencimiento
T = 252
dt = 1 / T

num_simulations = 10000

# Choques aleatorios para rendimientos del subyacente
Z = np.random.normal(0, 1, (num_simulations, T ) )

# Matriz de precios simulados del subyacente
# N filas X Pasos + 1 (Por S0)
price_matrix = np.zeros( (num_simulations, T + 1 ) )


In [ ]:
## PRICING

barrier_price = S0


for i in range (num_simulations):

    #Inicio de la trayectoria simulada
    price_path = [S0]
    barrier_breached = False

    for dia in range(1, T+1):
        St = price_path[-1] * np.exp((r - 0.5 * sigma ** 2) * dt + sigma * np.sqrt(dt) * Z[i, dia - 1])
        price_path.append(St)

        # Para este ejemplo, si ST es mayor a la barrera:
        if St > barrier_price:
            barrier_breached = True
            break

    if not barrier_breached:
        price_matrix[i, :] = price_path

    